<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ml801/dataset-filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading Dataset

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("HamdanXI/arb-eng-parallel-10k-splitted")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Cosine Similarity / Euclidean Distance

In [3]:
!pip install sentence-transformers scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=808f2a99e3a38fe225b88c68cdb059dac74b7d154f9fa7dd8633bd23192b418d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from datasets import load_dataset
from tqdm import tqdm

# Load tokenizer and model (e.g., multilingual BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

def embed_sentence(sentence, lang):
    tokens = tokenizer(sentence, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
    with torch.no_grad():
        embeddings = model(**tokens)
    return embeddings.last_hidden_state.mean(dim=1)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [5]:
# Initialize variables to store the maximum lengths
max_length_arabic = 0
max_length_english = 0

# Iterate over the dataset to find the maximum lengths
for row in dataset['train']:
    length_arabic = len(row['arabic'].split())  # Number of words in the Arabic sentence
    length_english = len(row['english'].split())  # Number of words in the English sentence

    # Update the maximum lengths if current lengths are greater
    if length_arabic > max_length_arabic:
        max_length_arabic = length_arabic
    if length_english > max_length_english:
        max_length_english = length_english

if max_length_arabic > max_length_english:
  max_length = max_length_arabic
else:
  max_length = max_length_english

print(max_length)

449


In [6]:
def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2)[0][0]

def calculate_euclidean_distance(embedding1, embedding2):
    return euclidean(embedding1, embedding2)[0][0]

In [7]:
# Initialize an empty list to store filtered data
filtered_data_cosine = []

# Define a similarity threshold
similarity_threshold = 0.7  # Adjust this based on your needs

for row in tqdm(dataset['train'], desc="Processing", unit="row"):
    arabic_embedding = embed_sentence(row['arabic'], 'ar')
    english_embedding = embed_sentence(row['english'], 'en')

    similarity = calculate_cosine_similarity(arabic_embedding, english_embedding)
    if similarity > similarity_threshold:
        filtered_data_cosine.append(row)

Processing: 100%|██████████| 8000/8000 [14:51<00:00,  8.98row/s]


In [8]:
# Initialize an empty list to store filtered data
filtered_data_eucloidean = []

# Define a similarity threshold
distance_threshold = 1.2  # Adjust this based on your needs

for row in tqdm(dataset['train'], desc="Processing", unit="row"):
    arabic_embedding = embed_sentence(row['arabic'], 'ar')
    english_embedding = embed_sentence(row['english'], 'en')

    distance = calculate_euclidean_distance(arabic_embedding, english_embedding)
    if distance <= distance_threshold:
        filtered_data_eucloidean.append(row)

Processing:   0%|          | 0/8000 [00:00<?, ?row/s]


ValueError: ignored

In [11]:
from datasets import Dataset

# Assuming filtered_data is a list of dictionaries
# Convert the filtered data to a dictionary format
data_dict = {"arabic": [row['arabic'] for row in filtered_data_cosine],
             "english": [row['english'] for row in filtered_data_cosine]}

# Create a new dataset from the dictionary
filtered_dataset = Dataset.from_dict(data_dict)

# Now filtered_dataset is a Hugging Face Dataset containing your filtered data

In [12]:
filtered_dataset

Dataset({
    features: ['arabic', 'english'],
    num_rows: 482
})